# Multicultural Fusion Cuisine Generator & Palate Predictor

This notebook provides an interactive demo of the complete fusion cuisine generation pipeline.

## Pipeline Overview:
1. **Data Loading**: Generate mock RecipeNLG dataset with Japanese and Italian recipes
2. **Preprocessing**: Clean data and create PyG-compatible format
3. **Model Training**: Train encoder and PalateNet models
4. **Fusion Generation**: Create fusion recipes with different alpha values
5. **Evaluation**: Analyze results and compute metrics

---

## Setup and Configuration

In [ ]:
import sys
import os
from pathlib import Path
import time
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, Image as IPImage
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Import configuration
from app.config import config
from app.utils import setup_logging

# Setup logging and display configuration
setup_logging("INFO")
print(f"Project root: {project_root}")
print(f"Configuration file: {config.config_path}")
print(f"Offline mode: {config.offline_mode}")
print(f"Random seed: {config.random_seed}")

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## Step 1: Data Loading and Generation

In [ ]:
print("🔄 Step 1: Data Loading and Generation")
print("=" * 50)

start_time = time.time()

# Run data fetching script
os.chdir(project_root)
%run scripts/01_fetch_data.py

data_time = time.time() - start_time
print(f"\n✅ Data generation completed in {data_time:.1f} seconds")

# Load and display data summary
try:
    with open(config.data_dir / "data_summary.json", "r") as f:
        data_summary = json.load(f)
    
    print("\n📊 Data Summary:")
    for key, value in data_summary.items():
        print(f"   {key}: {value}")
        
except Exception as e:
    print(f"Could not load data summary: {e}")


### Data Analysis and Visualization

In [ ]:
# Load and analyze the generated data
try:
    recipes_df = pd.read_csv(config.data_dir / "recipes.csv")
    ratings_df = pd.read_csv(config.data_dir / "ratings.csv")
    
    print(f"📈 Dataset Analysis:")
    print(f"   Total recipes: {len(recipes_df)}")
    print(f"   Total ratings: {len(ratings_df)}")
    print(f"   Average rating: {recipes_df['rating'].mean():.2f}")
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Recipe distribution by cuisine
    recipes_df['cuisine'].value_counts().plot(kind='bar', ax=axes[0,0], title='Recipes by Cuisine')
    axes[0,0].set_ylabel('Number of Recipes')
    
    # Rating distribution
    recipes_df['rating'].hist(bins=20, ax=axes[0,1], title='Rating Distribution')
    axes[0,1].set_xlabel('Rating')
    axes[0,1].set_ylabel('Frequency')
    
    # Ingredients per recipe
    recipes_df['num_ingredients'].hist(bins=15, ax=axes[1,0], title='Ingredients per Recipe')
    axes[1,0].set_xlabel('Number of Ingredients')
    axes[1,0].set_ylabel('Frequency')
    
    # Rating by cuisine
    recipes_df.boxplot(column='rating', by='cuisine', ax=axes[1,1])
    axes[1,1].set_title('Rating Distribution by Cuisine')
    
    plt.tight_layout()
    plt.show()
    
    # Show sample recipes
    print("\n🍽️  Sample Recipes:")
    for cuisine in config.target_cuisines:
        sample = recipes_df[recipes_df['cuisine'] == cuisine].iloc[0]
        print(f"\n{cuisine.upper()}:")
        print(f"  Title: {sample['title']}")
        print(f"  Ingredients: {sample['ingredients'][:100]}...")
        print(f"  Rating: {sample['rating']}")
        
except Exception as e:
    print(f"Error analyzing data: {e}")


## Step 2: Data Preprocessing

In [ ]:
print("\n🔄 Step 2: Data Preprocessing")
print("=" * 50)

start_time = time.time()

# Run preprocessing script
os.chdir(project_root)
%run scripts/02_preprocess.py

preprocess_time = time.time() - start_time
print(f"\n✅ Preprocessing completed in {preprocess_time:.1f} seconds")

# Load and display preprocessing metadata
try:
    with open(config.data_dir / "preprocessing_metadata.json", "r") as f:
        metadata = json.load(f)
    
    print("\n📊 Preprocessing Results:")
    for key, value in metadata.items():
        print(f"   {key}: {value}")
        
except Exception as e:
    print(f"Could not load preprocessing metadata: {e}")


### Preprocessing Analysis

In [ ]:
# Analyze preprocessing results
try:
    # Load processed data splits
    train_df = pd.read_csv(config.data_dir / "train.csv")
    val_df = pd.read_csv(config.data_dir / "val.csv")
    test_df = pd.read_csv(config.data_dir / "test.csv")
    
    # Load embeddings
    text_embeddings = np.load(config.data_dir / "text_embeddings.npy")
    ingredient_embeddings = np.load(config.data_dir / "ingredient_embeddings.npy")
    
    print("📈 Preprocessing Analysis:")
    print(f"   Train samples: {len(train_df)}")
    print(f"   Validation samples: {len(val_df)}")
    print(f"   Test samples: {len(test_df)}")
    print(f"   Text embedding dimension: {text_embeddings.shape[1]}")
    print(f"   Ingredient embedding dimension: {ingredient_embeddings.shape[1]}")
    
    # Visualize data splits
    split_data = {
        'Train': len(train_df),
        'Validation': len(val_df),
        'Test': len(test_df)
    }
    
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.pie(split_data.values(), labels=split_data.keys(), autopct='%1.1f%%')
    plt.title('Dataset Splits')
    
    # Visualize embedding dimensions
    plt.subplot(1, 2, 2)
    embedding_dims = {
        'Text': text_embeddings.shape[1],
        'Ingredients': ingredient_embeddings.shape[1]
    }
    plt.bar(embedding_dims.keys(), embedding_dims.values())
    plt.title('Embedding Dimensions')
    plt.ylabel('Dimension')
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error analyzing preprocessing results: {e}")


## Step 3: Model Training

### 3a. Train Encoder Model

In [ ]:
print("\n🔄 Step 3a: Training Encoder Model")
print("=" * 50)

start_time = time.time()

# Run encoder training script
os.chdir(project_root)
%run scripts/train_encoder.py

encoder_time = time.time() - start_time
print(f"\n✅ Encoder training completed in {encoder_time:.1f} seconds")


### 3b. Train PalateNet Model

In [ ]:
print("\n🔄 Step 3b: Training PalateNet Model")
print("=" * 50)

start_time = time.time()

# Run PalateNet training script
os.chdir(project_root)
%run scripts/train_palatenet.py

palatenet_time = time.time() - start_time
print(f"\n✅ PalateNet training completed in {palatenet_time:.1f} seconds")


### Training Results Analysis

In [ ]:
# Check if models were saved successfully
model_files = {
    'Encoder (Best)': config.models_dir / "encoder_best.pt",
    'Encoder (Final)': config.models_dir / "encoder_final.pt",
    'PalateNet (Best)': config.models_dir / "palatenet_best.pt",
    'PalateNet (Final)': config.models_dir / "palatenet_final.pt"
}

print("💾 Saved Models:")
for name, path in model_files.items():
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"   ✅ {name}: {size_mb:.1f} MB")
    else:
        print(f"   ❌ {name}: Not found")

print(f"\n⏱️  Total Training Time: {encoder_time + palatenet_time:.1f} seconds")


## Step 4: Fusion Generation

In [ ]:
print("\n🔄 Step 4: Fusion Generation")
print("=" * 50)

start_time = time.time()

# Run fusion generation script
os.chdir(project_root)
%run scripts/generate_fusion.py

generation_time = time.time() - start_time
print(f"\n✅ Fusion generation completed in {generation_time:.1f} seconds")


### Generated Fusion Recipes

In [ ]:
# Display generated fusion recipes
try:
    fusion_recipes_path = config.outputs_dir / "recipes" / "fusion_recipes.json"
    
    if fusion_recipes_path.exists():
        with open(fusion_recipes_path, "r") as f:
            fusion_recipes = json.load(f)
        
        print("🍽️  Generated Fusion Recipes:")
        print("=" * 60)
        
        for alpha_key, recipe_data in fusion_recipes.items():
            alpha = recipe_data["alpha"]
            recipe = recipe_data["recipe"]
            
            print(f"\n🎯 Alpha = {alpha} (Japanese: {alpha:.1f}, Italian: {1-alpha:.1f})")
            print("-" * 50)
            print(recipe[:500] + "..." if len(recipe) > 500 else recipe)
            print("\n" + "=" * 60)
    else:
        print("❌ No fusion recipes found")
        
except Exception as e:
    print(f"Error loading fusion recipes: {e}")


### Generated Images (if available)

In [ ]:
# Display generated images if available
try:
    fusion_images_path = config.outputs_dir / "images" / "fusion_images.json"
    
    if fusion_images_path.exists():
        with open(fusion_images_path, "r") as f:
            fusion_images = json.load(f)
        
        print("🖼️  Generated Fusion Images:")
        
        # Display images in a grid
        num_images = len(fusion_images)
        if num_images > 0:
            cols = min(3, num_images)
            rows = (num_images + cols - 1) // cols
            
            fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
            if rows == 1:
                axes = [axes] if cols == 1 else axes
            elif cols == 1:
                axes = [[ax] for ax in axes]
            
            for i, (alpha_key, image_data) in enumerate(fusion_images.items()):
                row, col = i // cols, i % cols
                ax = axes[row][col] if rows > 1 else axes[col]
                
                try:
                    from PIL import Image as PILImage
                    image_path = Path(image_data["file"])
                    if image_path.exists():
                        img = PILImage.open(image_path)
                        ax.imshow(img)
                        ax.set_title(f"Alpha = {image_data['alpha']}")
                        ax.axis('off')
                    else:
                        ax.text(0.5, 0.5, 'Image not found', ha='center', va='center')
                        ax.set_title(f"Alpha = {image_data['alpha']}")
                except Exception as e:
                    ax.text(0.5, 0.5, f'Error: {e}', ha='center', va='center')
                    ax.set_title(f"Alpha = {image_data['alpha']}")
            
            # Hide unused subplots
            for i in range(num_images, rows * cols):
                row, col = i // cols, i % cols
                ax = axes[row][col] if rows > 1 else axes[col]
                ax.axis('off')
            
            plt.tight_layout()
            plt.show()
        else:
            print("   No images were generated")
    else:
        print("📝 Images not generated (likely using text-only mode)")
        
except Exception as e:
    print(f"Error displaying images: {e}")


## Step 5: Evaluation

In [ ]:
print("\n🔄 Step 5: Comprehensive Evaluation")
print("=" * 50)

start_time = time.time()

# Run evaluation script
os.chdir(project_root)
%run scripts/evaluate.py

evaluation_time = time.time() - start_time
print(f"\n✅ Evaluation completed in {evaluation_time:.1f} seconds")


### Detailed Evaluation Results

In [ ]:
# Load and analyze evaluation results
try:
    evaluation_path = config.outputs_dir / "evaluation_metrics.json"
    
    if evaluation_path.exists():
        with open(evaluation_path, "r") as f:
            metrics = json.load(f)
        
        # Create comprehensive visualization of results
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Rating prediction performance
        if metrics.get("rating_prediction"):
            rating_metrics = metrics["rating_prediction"]
            
            metric_names = ['Spearman Corr', 'R² Score']
            metric_values = [
                rating_metrics.get('spearman_correlation', 0),
                rating_metrics.get('r2_score', 0)
            ]
            
            bars = axes[0,0].bar(metric_names, metric_values)
            axes[0,0].set_title('Rating Prediction Performance')
            axes[0,0].set_ylim(0, 1)
            
            # Add target line for Spearman correlation
            axes[0,0].axhline(y=config.target_spearman_correlation, color='r', linestyle='--', 
                            label=f'Target ({config.target_spearman_correlation})')
            axes[0,0].legend()
            
            # Color bars based on target achievement
            if rating_metrics.get('spearman_correlation', 0) >= config.target_spearman_correlation:
                bars[0].set_color('green')
            else:
                bars[0].set_color('orange')
        
        # Ingredient overlap analysis
        if metrics.get("ingredient_overlap"):
            overlap_metrics = metrics["ingredient_overlap"]
            alpha_overlaps = overlap_metrics.get('alpha_specific_overlaps', {})
            
            if alpha_overlaps:
                alphas = [float(k.split('_')[1]) for k in alpha_overlaps.keys()]
                overlaps = list(alpha_overlaps.values())
                
                axes[0,1].plot(alphas, overlaps, 'bo-', linewidth=2, markersize=8)
                axes[0,1].set_title('Ingredient Overlap by Alpha')
                axes[0,1].set_xlabel('Alpha (Japanese weight)')
                axes[0,1].set_ylabel('Overlap Score')
                axes[0,1].grid(True, alpha=0.3)
        
        # Diversity metrics
        if metrics.get("diversity"):
            diversity_metrics = metrics["diversity"]
            
            diversity_data = {
                'Unique\nIngredients': diversity_metrics.get('total_unique_ingredients', 0),
                'Avg Ingredients\nper Recipe': diversity_metrics.get('avg_ingredients_per_recipe', 0),
                'Fusion\nRecipes': diversity_metrics.get('num_fusion_recipes', 0)
            }
            
            bars = axes[1,0].bar(diversity_data.keys(), diversity_data.values())
            axes[1,0].set_title('Recipe Diversity Metrics')
            axes[1,0].tick_params(axis='x', rotation=45)
        
        # Pipeline performance summary
        pipeline_times = {
            'Data Gen': data_time,
            'Preprocessing': preprocess_time,
            'Encoder Train': encoder_time,
            'PalateNet Train': palatenet_time,
            'Generation': generation_time,
            'Evaluation': evaluation_time
        }
        
        axes[1,1].pie(pipeline_times.values(), labels=pipeline_times.keys(), autopct='%1.1f%%')
        axes[1,1].set_title('Pipeline Time Distribution')
        
        plt.tight_layout()
        plt.show()
        
        # Summary statistics
        total_time = sum(pipeline_times.values())
        
        print("\n📊 PIPELINE PERFORMANCE SUMMARY")
        print("=" * 50)
        print(f"Total Pipeline Time: {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
        
        if metrics.get("rating_prediction"):
            spearman = metrics["rating_prediction"].get('spearman_correlation', 0)
            target_achieved = spearman >= config.target_spearman_correlation
            print(f"Target Achievement: {'✅ SUCCESS' if target_achieved else '❌ FAILED'}")
            print(f"Spearman Correlation: {spearman:.4f} (target: {config.target_spearman_correlation})")
        
        if metrics.get("ingredient_overlap"):
            avg_overlap = metrics["ingredient_overlap"].get('avg_ingredient_overlap', 0)
            print(f"Average Ingredient Overlap: {avg_overlap:.4f}")
        
        print(f"Fusion Recipes Generated: {len(fusion_recipes) if 'fusion_recipes' in locals() else 0}")
        
    else:
        print("❌ Evaluation metrics not found")
        
except Exception as e:
    print(f"Error analyzing evaluation results: {e}")


## Summary and Conclusions

In [ ]:
print("\n🎉 FUSION CUISINE PIPELINE COMPLETE!")
print("=" * 60)

# Calculate total pipeline time
total_pipeline_time = (
    data_time + preprocess_time + encoder_time + 
    palatenet_time + generation_time + evaluation_time
)

print(f"\n⏱️  TIMING BREAKDOWN:")
print(f"   Data Generation: {data_time:.1f}s ({data_time/total_pipeline_time*100:.1f}%)")
print(f"   Preprocessing: {preprocess_time:.1f}s ({preprocess_time/total_pipeline_time*100:.1f}%)")
print(f"   Encoder Training: {encoder_time:.1f}s ({encoder_time/total_pipeline_time*100:.1f}%)")
print(f"   PalateNet Training: {palatenet_time:.1f}s ({palatenet_time/total_pipeline_time*100:.1f}%)")
print(f"   Fusion Generation: {generation_time:.1f}s ({generation_time/total_pipeline_time*100:.1f}%)")
print(f"   Evaluation: {evaluation_time:.1f}s ({evaluation_time/total_pipeline_time*100:.1f}%)")
print(f"   TOTAL: {total_pipeline_time:.1f}s ({total_pipeline_time/60:.1f} minutes)")

print(f"\n💾 OUTPUT FILES:")
output_files = [
    config.outputs_dir / "recipes" / "fusion_recipes.json",
    config.outputs_dir / "evaluation_metrics.json",
    config.outputs_dir / "fusion_embeddings.json"
]

for file_path in output_files:
    if file_path.exists():
        size_kb = file_path.stat().st_size / 1024
        print(f"   ✅ {file_path.name}: {size_kb:.1f} KB")
    else:
        print(f"   ❌ {file_path.name}: Not found")

# Check for generated images
images_dir = config.outputs_dir / "images"
if images_dir.exists():
    image_files = list(images_dir.glob("*.png")) + list(images_dir.glob("*.jpg"))
    print(f"   🖼️  Generated Images: {len(image_files)}")

print(f"\n🎯 SUCCESS CRITERIA:")
try:
    if 'metrics' in locals() and metrics.get("rating_prediction"):
        spearman = metrics["rating_prediction"].get('spearman_correlation', 0)
        target_met = spearman >= config.target_spearman_correlation
        print(f"   Spearman Correlation ≥ {config.target_spearman_correlation}: {'✅' if target_met else '❌'} ({spearman:.4f})")
    
    if 'fusion_recipes' in locals():
        recipes_generated = len(fusion_recipes) == len(config.fusion_alphas)
        print(f"   Fusion Recipes Generated: {'✅' if recipes_generated else '❌'} ({len(fusion_recipes)}/{len(config.fusion_alphas)})")
    
    time_limit_met = total_pipeline_time <= 4 * 3600  # 4 hours
    print(f"   Time Limit (≤4 hours): {'✅' if time_limit_met else '❌'} ({total_pipeline_time/3600:.2f}h)")
    
except Exception as e:
    print(f"   Error checking success criteria: {e}")

print(f"\n📁 All outputs saved to: {config.outputs_dir}")
print(f"📝 Models saved to: {config.models_dir}")
print(f"📊 Data saved to: {config.data_dir}")

print("\n🚀 Pipeline execution completed successfully!")
print("   You can now explore the generated fusion recipes and evaluation results.")
print("   Try running individual components again with different parameters.")


## Interactive Exploration

Use the cells below to explore the results interactively:

In [ ]:
# Interactive: Explore specific fusion recipes
try:
    if 'fusion_recipes' in locals():
        print("🔍 Interactive Recipe Explorer")
        print("Available alpha values:", [data['alpha'] for data in fusion_recipes.values()])
        
        # You can modify this to explore specific recipes
        alpha_to_explore = 0.5  # Change this value
        
        for key, data in fusion_recipes.items():
            if abs(data['alpha'] - alpha_to_explore) < 0.01:
                print(f"\n📖 Fusion Recipe (α={data['alpha']}):")
                print("=" * 50)
                print(data['recipe'])
                break
        else:
            print(f"No recipe found for alpha = {alpha_to_explore}")
except:
    print("Fusion recipes not available")


In [ ]:
# Interactive: Model performance analysis
try:
    if 'metrics' in locals():
        print("📈 Model Performance Deep Dive")
        print("=" * 40)
        
        # Rating prediction details
        if metrics.get('rating_prediction'):
            rating_perf = metrics['rating_prediction']
            print(f"\n🎯 Rating Prediction Model:")
            print(f"   Spearman Correlation: {rating_perf.get('spearman_correlation', 0):.4f}")
            print(f"   RMSE: {rating_perf.get('rmse', 0):.4f}")
            print(f"   MAE: {rating_perf.get('mae', 0):.4f}")
            print(f"   R² Score: {rating_perf.get('r2_score', 0):.4f}")
            print(f"   Test Samples: {rating_perf.get('num_samples', 0)}")
        
        # Ingredient overlap details
        if metrics.get('ingredient_overlap'):
            overlap_perf = metrics['ingredient_overlap']
            print(f"\n🥘 Ingredient Overlap Analysis:")
            print(f"   Average Overlap: {overlap_perf.get('avg_ingredient_overlap', 0):.4f}")
            print(f"   Standard Deviation: {overlap_perf.get('std_ingredient_overlap', 0):.4f}")
            
            alpha_overlaps = overlap_perf.get('alpha_specific_overlaps', {})
            for alpha_key, overlap in alpha_overlaps.items():
                print(f"   {alpha_key}: {overlap:.4f}")
                
except:
    print("Metrics not available for analysis")


---

## 🎉 Congratulations!

You have successfully completed the **Multicultural Fusion Cuisine Generator & Palate Predictor** pipeline!

### What was accomplished:
1. ✅ Generated mock RecipeNLG dataset with 10,000 recipes (5K Japanese + 5K Italian)
2. ✅ Created ingredient FlavorGraph with co-occurrence relationships  
3. ✅ Preprocessed data for PyTorch Geometric compatibility
4. ✅ Trained CLIP-mini encoder for text/image fusion
5. ✅ Trained GraphSAGE + MLP PalateNet for rating prediction
6. ✅ Generated fusion recipes with different α values (0.3, 0.5, 0.7)
7. ✅ Evaluated system performance with comprehensive metrics

### Key Results:
- **Rating Prediction**: Achieved Spearman correlation for taste prediction
- **Fusion Generation**: Created novel Japanese-Italian fusion recipes
- **Ingredient Analysis**: Analyzed ingredient overlap and diversity
- **Pipeline Performance**: Complete end-to-end execution within resource constraints

### Next Steps:
- Experiment with different α values for fusion
- Try the pipeline with your own recipe data
- Explore the generated recipes and adapt them for cooking
- Enhance the models with additional cuisines or features

**Happy cooking and experimenting! 👨‍🍳👩‍🍳**